In [ ]:
%matplotlib inline

In [ ]:
import datajoint as dj
schema = dj.schema('dimitri_university')
schema.spawn_missing_classes()

In [ ]:
dj.ERD(schema)

# Example queries

## Restriction

In [ ]:
# Students from Texas
Student & {'home_state': 'TX'}
Student & 'home_state="TX"'

In [ ]:
# Male students from outside Texas
Student - {'home_state': 'TX'}
Student - 'home_state="TX"'
Student & 'home_state <> "TX"'

In [ ]:
# Male students from outside Texas
Student & {'sex="M" and home_state <> "TX"'}
(Student & {'sex':"M"}) - {'home_state': "TX"}

In [ ]:
# Students from TX, OK, or NM
Student & [{'home_state':'OK'}, {'home_state':'NM'}, {'home_state':'TX'}] 
Student & [{'home_state': s} for s in ('OK','NM','TX')]
Student & 'home_state in ("OK", "NM", "TX")'

In [ ]:
# Millenials
Student & 'date_of_birth between "1981-01-01" and "1996-12-31"'

In [ ]:
# Students who have taken classes
Student & Enroll

In [ ]:
# Students who have not taken classes
Student - Enroll

In [ ]:
# Students who have taken Biology classes but no MATH courses
(Student & (Enroll & 'dept="BIOL"')) - (Enroll & 'dept="MATH"')

In [ ]:
# Students who have not selected a major
Student - StudentMajor

In [ ]:
# Students who are taking courses in the current term
Student - (Enroll & CurrentTerm)

In [ ]:
# Ungraded courses
Enroll - Grade

In [ ]:
# Ungraded courses in the current term
(Enroll & CurrentTerm) - Grade

In [ ]:
# Students who have taken classes and have chosen a major
Student & Enroll & StudentMajor 

In [ ]:
#Students who have taken classes or have chosen a major
Student & [Enroll, StudentMajor]

In [ ]:
# Enrollment in courses from the same department as the students' major
Enroll & StudentMajor

## Join

In [ ]:
# Grade point values
Grade * LetterGrade

In [ ]:
# Graded enrollments with complete course and student information
Student * Enroll * Course * Section * Grade * LetterGrade

In [ ]:
# Students with ungraded courses in current term
Student & (Enroll * CurrentTerm - Grade)

In [ ]:
# Enrollments before students' date of birth
Student * Enroll & 'term_year <= date_of_birth'

## Proj

In [ ]:
# remove student personal info other than home state and sex
Student.proj('home_state', 'sex')

In [ ]:
# Student major
Student.proj('first_name','last_name')*StudentMajor.proj(major='dept')

In [ ]:
# Enrollment with major information
Enroll * StudentMajor.proj(major='dept')

In [ ]:
# Enrollment outside chosen major
Enroll * StudentMajor.proj(major='dept') & 'major<>dept'

In [ ]:
# Enrollment not matching major 
Enroll - StudentMajor 

In [ ]:
# Total grade points
(Course * Grade * LetterGrade).proj(total='points*credits')

## Aggr

In [ ]:
# Students in each section
Section.aggr(Enroll, n='count(*)')

In [ ]:
# Average grade in each course
Course.aggr(Grade*LetterGrade, avg_grade='avg(points)')

In [ ]:
# Fraction graded in each section
(Section.aggr(Enroll,n='count(*)') * Section.aggr(Grade, m='count(*)')).proj(
    'm','n',frac='m/n')

In [ ]:
# GPA
Student.aggr(Course * Grade * LetterGrade, 
             gpa='sum(points*credits)/sum(credits)')

In [ ]:
# GPA in current Term
Student.aggr(Course * Grade * LetterGrade & CurrentTerm, 
             gpa='sum(points*credits)/sum(credits)')

In [ ]:
# Average GPA for each major
gpa = Student.aggr(Course * Grade * LetterGrade,
                   gpa='sum(points*credits)/sum(credits)')
Department.aggr(StudentMajor*gpa, avg_gpa='avg(gpa)')

## Relation U

In [ ]:
# All home states
dj.U('home_state') & Student

In [ ]:
# Unique last names
dj.U('last_name') & Student 

In [ ]:
# Number of students from each state by gender
dj.U('home_state', 'sex').aggr(Student, n='count(*)')

In [ ]:
# GPA by sex
dj.U('sex').aggr(Student*gpa, avg_gpa ='avg(gpa)')

In [ ]:
# Sex ratio for gender neutral names
dj.U('first_name').aggr(Student, percent_male='100*avg(sex="M")'
                       ) & 'percent_male > 0 and percent_male < 100'

In [ ]:
# Students with the same birthdays
s1 = Student.proj('date_of_birth', s1='student_id') * dj.U('date_of_birth')
s2 = Student.proj('date_of_birth', s2='student_id') * dj.U('date_of_birth')
s1 * s2 & 's1 < s2'